In [3]:
import torch
import numpy as np
import pandas as pd

from torch.utils.data import Dataset
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score, roc_auc_score
from transformers import AutoTokenizer, AutoModelForSequenceClassification, Trainer, TrainingArguments, EarlyStoppingCallback
from transformers import AutoModel


In [8]:
# Kiểm tra GPU
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [9]:
# Tải tokenizer và mô hình
tokenizer = AutoTokenizer.from_pretrained("vinai/phobert-base")
model = AutoModel.from_pretrained("vinai/phobert-base").to(device)

In [10]:
# Đọc dữ liệu
train_df = pd.read_csv('train_data.csv')
val_df = pd.read_csv('val_data.csv')

train_texts = train_df['content'].tolist()
train_labels = np.array(train_df['label']).astype(int)
val_texts = val_df['content'].tolist()
val_labels = np.array(val_df['label']).astype(int)

In [11]:
# Tokenize dữ liệu với padding
def tokenize_and_pad(texts, tokenizer, max_length=256):
    return tokenizer(
        texts,
        truncation=True,
        padding='max_length',
        max_length=max_length
    )

train_encodings = tokenize_and_pad(train_texts, tokenizer)
val_encodings = tokenize_and_pad(val_texts, tokenizer)

In [12]:
# Kiểm tra dữ liệu sau khi token hóa
print(train_encodings.keys())
print(val_encodings.keys())
print(train_encodings['input_ids'][:2])
print(val_encodings['input_ids'][:2])

dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
dict_keys(['input_ids', 'token_type_ids', 'attention_mask'])
[[0, 1656, 8, 1347, 8915, 336, 5963, 2546, 620, 396, 30, 1302, 9412, 56669, 11, 197, 133, 151, 3634, 848, 99, 396, 123, 292, 336, 20014, 2, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [0, 2925, 474, 2515, 23523, 34, 275, 262, 829, 133

In [13]:
# Tạo dataset
class FakeNewsDataset(Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]).to(device) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx]).to(device).squeeze()
        return item

    def __len__(self):
        return len(self.labels)

In [14]:
train_dataset = FakeNewsDataset(train_encodings, train_labels.tolist())
val_dataset = FakeNewsDataset(val_encodings, val_labels.tolist())

In [15]:
# Tải mô hình phân loại
model = AutoModelForSequenceClassification.from_pretrained("vinai/phobert-base", num_labels=2)

Some weights of RobertaForSequenceClassification were not initialized from the model checkpoint at vinai/phobert-base and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


In [16]:
def train_model(learning_rate, weight_decay, num_train_epochs):
    training_args = TrainingArguments(
        output_dir='./results',
        num_train_epochs=num_train_epochs,
        per_device_train_batch_size=8,
        per_device_eval_batch_size=8,
        warmup_steps=500,
        weight_decay=weight_decay,
        logging_dir='./logs',
        logging_steps=10,
        evaluation_strategy="steps",
        eval_steps=50,
        save_steps=50,
        save_total_limit=3,
        load_best_model_at_end=True,
        learning_rate=learning_rate,
    )

    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset,
        eval_dataset=val_dataset,
    )

    trainer.train()

    eval_result = trainer.evaluate()
    return eval_result['eval_loss']

In [17]:
# Tinh chỉnh siêu tham số với Grid Search
param_grid = {
    'learning_rate': [5e-5, 3e-5, 2e-5],
    'weight_decay': [0.01, 0.015, 0.02],
    'num_train_epochs': [3, 4, 5],
}

best_params = None
best_score = float('inf')

for lr in param_grid['learning_rate']:
    for wd in param_grid['weight_decay']:
        for epochs in param_grid['num_train_epochs']:
            eval_loss = train_model(lr, wd, epochs)
            print(f"Learning rate: {lr}, Weight decay: {wd}, Epochs: {epochs}, Eval loss: {eval_loss}")
            if eval_loss < best_score:
                best_score = eval_loss
                best_params = {'learning_rate': lr, 'weight_decay': wd, 'num_train_epochs': epochs}

print(f"Best params: {best_params}, Best eval loss: {best_score}")

c:\Users\PC\AppData\Local\Programs\Python\Python310\lib\site-packages\transformers\training_args.py:1525: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


  0%|          | 0/423 [00:00<?, ?it/s]

{'loss': 0.7129, 'grad_norm': 3.921420097351074, 'learning_rate': 1.0000000000000002e-06, 'epoch': 0.07}
{'loss': 0.6952, 'grad_norm': 3.9515187740325928, 'learning_rate': 2.0000000000000003e-06, 'epoch': 0.14}
{'loss': 0.6927, 'grad_norm': 4.282040119171143, 'learning_rate': 3e-06, 'epoch': 0.21}
{'loss': 0.6708, 'grad_norm': 2.932995319366455, 'learning_rate': 4.000000000000001e-06, 'epoch': 0.28}
{'loss': 0.6709, 'grad_norm': 6.047645568847656, 'learning_rate': 5e-06, 'epoch': 0.35}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 0.6601576805114746, 'eval_runtime': 212.5339, 'eval_samples_per_second': 1.327, 'eval_steps_per_second': 0.169, 'epoch': 0.35}
{'loss': 0.663, 'grad_norm': 3.6535658836364746, 'learning_rate': 6e-06, 'epoch': 0.43}
{'loss': 0.64, 'grad_norm': 3.5825765132904053, 'learning_rate': 7.000000000000001e-06, 'epoch': 0.5}
{'loss': 0.6077, 'grad_norm': 4.021620750427246, 'learning_rate': 8.000000000000001e-06, 'epoch': 0.57}
{'loss': 0.6546, 'grad_norm': 6.027839183807373, 'learning_rate': 9e-06, 'epoch': 0.64}
{'loss': 0.5909, 'grad_norm': 3.8980185985565186, 'learning_rate': 1e-05, 'epoch': 0.71}


...

{'eval_loss': 2.0817503929138184, 'eval_runtime': 176.1963, 'eval_samples_per_second': 1.6, 'eval_steps_per_second': 0.204, 'epoch': 1.06}
{'loss': 0.0, 'grad_norm': 7.0417520703358605e-09, 'learning_rate': 9.600000000000001e-06, 'epoch': 1.13}
{'loss': 0.0, 'grad_norm': 8.766122050474223e-09, 'learning_rate': 1.02e-05, 'epoch': 1.21}
{'loss': 0.0, 'grad_norm': 6.729522716852898e-09, 'learning_rate': 1.08e-05, 'epoch': 1.28}
{'loss': 0.0, 'grad_norm': 5.258500745952688e-09, 'learning_rate': 1.1400000000000001e-05, 'epoch': 1.35}
{'loss': 0.0, 'grad_norm': 5.576621830982731e-09, 'learning_rate': 1.2e-05, 'epoch': 1.42}


  0%|          | 0/36 [00:00<?, ?it/s]

{'eval_loss': 2.082066774368286, 'eval_runtime': 175.065, 'eval_samples_per_second': 1.611, 'eval_steps_per_second': 0.206, 'epoch': 1.42}
{'loss': 0.0, 'grad_norm': 7.286288461472168e-09, 'learning_rate': 1.26e-05, 'epoch': 1.49}
{'loss': 0.0, 'grad_norm': 5.166517880184074e-09, 'learning_rate': 1.32e-05, 'epoch': 1.56}
{'loss': 0.0, 'grad_norm': 5.44517764211605e-09, 'learning_rate': 1.3800000000000002e-05, 'epoch': 1.63}
{'loss': 0.0, 'grad_norm': 7.714430871885725e-09, 'learning_rate': 1.44e-05, 'epoch': 1.7}
{'loss': 0.0, 'grad_norm': 6.871472280067792e-09, 'learning_rate': 1.5e-05, 'epoch': 1.77}


  0%|          | 0/36 [00:00<?, ?it/s]

KeyboardInterrupt: 

In [ ]:
# Huấn luyện với tham số tốt nhất
training_args_with_early_stop = TrainingArguments(
    output_dir='./results_with_early_stop',
    num_train_epochs=best_params['num_train_epochs'],
    per_device_train_batch_size=8,
    per_device_eval_batch_size=8,
    warmup_steps=500,
    weight_decay=best_params['weight_decay'],
    logging_dir='./logs_with_early_stop',
    logging_steps=10,
    eval_strategy="steps",  # Đổi từ evaluation_strategy thành eval_strategy
    eval_steps=50,
    save_steps=50,
    save_total_limit=3,
    load_best_model_at_end=True,
    learning_rate=best_params['learning_rate'],
)

In [ ]:
trainer_with_early_stop = Trainer(
    model=model,
    args=training_args_with_early_stop,
    train_dataset=train_dataset,
    eval_dataset=val_dataset,
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

In [ ]:
trainer_with_early_stop.train()

In [ ]:
# Dự đoán nhãn cho tập kiểm tra
predictions2 = trainer_with_early_stop.predict(val_dataset)

# Lấy nhãn dự đoán từ logits
pred_labels2 = np.argmax(predictions2.predictions, axis=1)

In [ ]:
print(pred_labels2[:])  # In ra dự đoán

In [ ]:
# Tính các chỉ số
accuracy2 = accuracy_score(val_labels, pred_labels2)
precision2 = precision_score(val_labels, pred_labels2, pos_label=0)
recall2 = recall_score(val_labels, pred_labels2, pos_label=0)
f12 = f1_score(val_labels, pred_labels2, pos_label=0)
auc2 = roc_auc_score(val_labels, predictions2.predictions[:, 1])

print(f"Accuracy: {accuracy2:.6f}")
print(f"Precision: {precision2:.6f}")
print(f"Recall: {recall2:.6f}")
print(f"F1 Score: {f12:.6f}")
print(f'AUC: {auc2:.6f}')